## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sci

from sklearn.metrics import confusion_matrix

## Exercise 9.11.)

Do a classification analysis on the rootstock data of Table 6.2 as follows:

1. Find the linear classification functions.
2. Find the classification table using the linear classification functions in part 1 (assuming $\Sigma_1 = \Sigma_2 = \Sigma_3$).
3. Find the classification table using quadratic classification functions (assuming population covariance matrices are not equal).
4. Find the classification table using the nearest neighbor method.
5. Find the classification table using a kernel density estimator 

### Setup

In [2]:
names = ["type"] + [f"y{i+1}" for i in range(4)]
df = pd.read_table("Data/T6_2_ROOT.DAT", header=None, sep="\s+", names=names)

In [3]:
df.head(10)

,type,y1,y2,y3,y4
0,1,1.11,2.569,3.58,0.760
1,1,1.19,2.928,3.75,0.821
2,1,1.09,2.865,3.93,0.928
3,1,1.25,3.844,3.94,1.009
4,1,1.11,3.027,3.60,0.766
5,1,1.08,2.336,3.51,0.726
6,1,1.11,3.211,3.98,1.209
7,1,1.16,3.037,3.62,0.750
8,2,1.05,2.074,4.09,1.036
9,2,1.17,2.885,4.06,1.094


In [4]:
type_l = list(set(df["type"]))
type_l

[1, 2, 3, 4, 5, 6]

In [5]:
df_typed = [df[df["type"] == i][["y1", "y2", "y3", "y4"]] for i in type_l]

In our case for all classes we have the same number of trees:

In [6]:
for df_i in df_typed:
    print(df_i.shape[0])

8
8
8
8
8
8


In [7]:
n = df_typed[0].shape[0]

In [8]:
S = []
for df_i in df_typed:
    S.append(df_i.cov().to_numpy())
S = np.array(S)

In [9]:
S_pl = (n-1)/( df.shape[0] - len(type_l) ) * np.sum(S, axis=0)

In [10]:
S_pl

array([[0.00761875, 0.04039437, 0.01319256, 0.00517   ],
       [0.04039437, 0.28911405, 0.10389554, 0.05024318],
       [0.01319256, 0.10389554, 0.1021622 , 0.05908705],
       [0.00517   , 0.05024318, 0.05908705, 0.04101249]])

In [11]:
y_mean = []
for df_i in df_typed:
    y_mean.append(df_i.mean().to_numpy())

y_mean = np.array(y_mean)

### 1-2

In [12]:
def L(y):
    ans = []
    for i in range(len(type_l)):
        ans.append(y_mean[i].T @ np.linalg.inv(S_pl) @ y - 0.5 * y_mean[i].T @ np.linalg.inv(S_pl) @ y_mean[i])
    return np.array(ans)

In [13]:
def linear_classify(y):
    return type_l[np.argmax(L(y))]

In [14]:
df["lin_class"] = df[["y1", "y2", "y3", "y4"]].apply(linear_classify, axis=1)

In [15]:
answer = confusion_matrix(df["type"], df["lin_class"])
print(answer)

[[5 0 0 1 0 2]
 [0 3 2 1 2 0]
 [0 0 6 1 1 0]
 [3 0 1 4 0 0]
 [0 3 1 0 3 1]
 [2 0 0 0 2 4]]


In [16]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.5208333333333334

### 3

In [17]:
def Q(y):
    ans = []
    for i in range(len(type_l)):
        ans.append(-0.5 * np.log(np.linalg.det(S[i])) - 0.5 * (y - y_mean[i]).T @ np.linalg.inv(S[i]) @ (y - y_mean[i]))
    return np.array(ans)

In [18]:
def quadratic_classify(y):
    return type_l[np.argmax(Q(y))]

In [19]:
df["quad_class"] = df[["y1", "y2", "y3", "y4"]].apply(quadratic_classify, axis=1)

In [20]:
answer = confusion_matrix(df["type"], df["quad_class"])
print(answer)

[[8 0 0 0 0 0]
 [0 7 0 1 0 0]
 [1 0 6 0 1 0]
 [0 0 1 7 0 0]
 [0 3 0 0 4 1]
 [2 0 0 0 1 5]]


In [21]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.7708333333333334

### 4

In [22]:
def dist(y1, y2):
    return (y1 - y2).T @ np.linalg.inv(S_pl) @ (y1 - y2)

In [23]:
def knn(y, k=3):
    dist_list = []
    for i in range(df.shape[0]):
        yi = df[["y1", "y2", "y3", "y4"]].to_numpy()[i]
        dist_list.append(dist(y, yi))
        
    nearest = np.argsort(np.array(dist_list))
    
    return nearest[:k]

In [24]:
def knn_classify(y, k=3):
    # equal n_i are assumed, as in our situation
    # in case of equal k_i we choose the nearest
    nearest = knn(y, k)
    type_nearest = df["type"][nearest]
    type_nearest = list(type_nearest)
    
    return max(set(type_nearest), key=type_nearest.count)

In [25]:
def knn_classify_with_ties(y, k=3):
    # collect information about ties in the process
    global ties_l
    
    nearest = knn(y, k)
    type_nearest = df["type"][nearest]
    type_nearest = list(type_nearest)
    
    apperances = np.array([type_nearest.count(type_l[i]) for i in range(len(type_l))])
    
    winner = np.argmax(apperances)
    if apperances[winner] == np.sort(apperances)[-2]:
        ties_l[type_nearest[0]-1] += 1
        
    return type_l[winner]

In [26]:
df["knn_class"] = df[["y1", "y2", "y3", "y4"]].apply(knn_classify, axis=1)

In [27]:
answer = confusion_matrix(df["type"], df["knn_class"])
print(answer)

[[6 0 0 2 0 0]
 [0 7 0 0 1 0]
 [1 0 6 0 1 0]
 [3 0 0 5 0 0]
 [0 1 0 0 6 1]
 [2 0 0 1 0 5]]


In [28]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.7291666666666666

In [29]:
ties_l = np.zeros(len(type_l))
df[["y1", "y2", "y3", "y4"]].apply(knn_classify_with_ties, axis=1)
ties_l

array([1., 3., 0., 3., 0., 1.])

### 5

In [30]:
def f_hat(y, h=1, p=4):
    # normal kernel
    
    coef = 1/(df.shape[0] * h**p * (np.linalg.det(S_pl))**0.5)
    
    ans = np.zeros(len(type_l))
    for i in range(len(type_l)):
        for yj in df_typed[i].to_numpy():
            ans[i] += np.exp(-(y - yj).T @ np.linalg.inv(S_pl) @ (y - yj) / (2 * h ** 2))
    
    return coef * ans

def kde_classify(y):
    return type_l[np.argmax(f_hat(y))]

In [31]:
df["kde_class"] = df[["y1", "y2", "y3", "y4"]].apply(kde_classify, axis=1)

In [32]:
answer = confusion_matrix(df["type"], df[["y1", "y2", "y3", "y4"]].apply(kde_classify, axis=1))
print(answer)

[[8 0 0 0 0 0]
 [0 8 0 0 0 0]
 [1 0 6 0 1 0]
 [1 0 0 7 0 0]
 [0 0 0 0 7 1]
 [2 0 0 0 0 6]]


In [33]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.875

In [34]:
df

,type,y1,y2,y3,y4,lin_class,quad_class,knn_class,kde_class
0,1,1.11,2.569,3.58,0.760,1,1,1,1
1,1,1.19,2.928,3.75,0.821,1,1,1,1
2,1,1.09,2.865,3.93,0.928,6,1,1,1
3,1,1.25,3.844,3.94,1.009,1,1,4,1
4,1,1.11,3.027,3.60,0.766,1,1,1,1
5,1,1.08,2.336,3.51,0.726,6,1,1,1
6,1,1.11,3.211,3.98,1.209,4,1,4,1
7,1,1.16,3.037,3.62,0.750,1,1,1,1
8,2,1.05,2.074,4.09,1.036,5,2,2,2
9,2,1.17,2.885,4.06,1.094,4,4,2,2
